In [ ]:
pip install --upgrade openpyxl

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

# Hyperparameters
latent_dim = 10
batch_size = 32
epochs = 1000
lr = 0.0002

# 로또 번호의 범위와 길이
lotto_num_range = 45
lotto_num_length = 6

# 실제 당첨 번호 데이터를 불러오는 함수 (bonus 번호 제외)
def load_real_lotto_data(file_path):
    df = pd.read_csv(file_path)
    data = df.iloc[:, :lotto_num_length].values  # 첫 6개의 열만 사용
    return torch.FloatTensor(data)

# Generator 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, lotto_num_length),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.model(z) * lotto_num_range  # 1~45 사이로 조정

# Discriminator 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(lotto_num_length, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# 모델 초기화
generator = Generator()
discriminator = Discriminator()

# 손실 함수와 최적화 함수
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

# 실제 당첨 번호 데이터 불러오기
real_data = load_real_lotto_data('/content/lotto.csv')

# GAN 학습
for epoch in range(epochs):
    # Discriminator 학습
    for _ in range(batch_size):
        # 진짜 데이터 샘플
        real_samples = real_data[torch.randint(0, real_data.size(0), (batch_size,))]
        real_labels = torch.ones((batch_size, 1))

        # 가짜 데이터 생성
        z = torch.randn(batch_size, latent_dim)
        fake_samples = generator(z)
        fake_labels = torch.zeros((batch_size, 1))

        # 진짜 데이터 손실 계산
        real_output = discriminator(real_samples)
        loss_real = criterion(real_output, real_labels)

        # 가짜 데이터 손실 계산
        fake_output = discriminator(fake_samples.detach())
        loss_fake = criterion(fake_output, fake_labels)

        # Discriminator 최적화
        loss_D = (loss_real + loss_fake) / 2
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

    # Generator 학습
    z = torch.randn(batch_size, latent_dim)
    generated_data = generator(z)
    output = discriminator(generated_data)
    loss_G = criterion(output, real_labels)

    optimizer_G.zero_grad()
    loss_G.backward()
    optimizer_G.step()

    # 학습 진행 출력
    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{epochs}]  Loss_D: {loss_D.item():.4f}  Loss_G: {loss_G.item():.4f}")

# 최종 생성된 로또 번호 예측
with torch.no_grad():
    z = torch.randn(1, latent_dim)
    generated_lotto_numbers = generator(z).numpy().flatten()
    generated_lotto_numbers = np.sort(np.round(generated_lotto_numbers))
    print("Generated Lotto Numbers:", generated_lotto_numbers)


RuntimeError: all elements of input should be between 0 and 1

In [ ]:
import torch
import random
import numpy as np  # np 모듈을 사용하므로 추가 필요

import torch.nn as nn
import torch.optim as optim

# DQN 모델 정의
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 및 학습 관련 변수 설정
input_dim = lotto_num_range  # 입력 차원: 로또 번호 범위 (45)
output_dim = lotto_num_range  # 출력 차원: 로또 번호 범위 (45)
dqn = DQN(input_dim, output_dim)
optimizer = optim.Adam(dqn.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
gamma = 0.99  # 할인율


episodes = 1000  # 학습 반복 횟수
lotto_num_range = 45  # 로또 번호 범위

# 환경 클래스 정의
class LottoEnv:
    def __init__(self, lotto_num_range=45, num_picks=6):
        self.lotto_num_range = lotto_num_range  # 로또 번호 범위 (1~45)
        self.num_picks = num_picks  # 고를 번호 개수

    def reset(self):
        # 환경 초기화: 무작위로 6개의 번호를 선택하여 상태로 반환
        self.state = random.sample(range(1, self.lotto_num_range + 1), self.num_picks)
        return self.state

    def step(self, action):
        # 에이전트의 행동(action) 결과와 보상 계산
        if action in self.state:
            reward = 1  # 맞춘 번호가 있으면 보상 1
            self.state.remove(action)  # 맞춘 번호 제거
        else:
            reward = -0.1  # 틀린 경우 벌점

        done = len(self.state) == 0  # 모두 맞추면 종료
        next_state = self.state  # 현재 상태 반환
        return next_state, reward, done

# 환경 객체 초기화
env = LottoEnv()

# 예측 함수
def predict_numbers(dqn, num_count=6, ticket_count=10):
    """학습한 DQN 모델을 사용해 복권 티켓 수만큼 각 6개의 번호 예측"""
    predictions = []
    for _ in range(ticket_count):
        # 모든 번호에 대해 입력 벡터 초기화 (1, 45)
        state_tensor = torch.zeros(1, lotto_num_range)  # (1, 45) 차원 벡터

        # 네트워크의 Q 값을 기반으로 상위 num_count개의 번호 예측
        with torch.no_grad():
            q_values = dqn(state_tensor)
            top_numbers = torch.topk(q_values, num_count, dim=1).indices.squeeze().tolist()

        # 번호는 1부터 시작하므로 1을 더해 조정
        predicted_numbers = [num + 1 for num in top_numbers]

        # 각 예측된 번호 조합을 저장
        predictions.append(predicted_numbers)

    return predictions

# 메인 학습 루프
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        state_vector = np.zeros(lotto_num_range)
        for num in state:
            state_vector[num - 1] = 1
        state_tensor = torch.FloatTensor(state_vector).unsqueeze(0)

        if random.random() < 0.1:
            action = random.randint(1, lotto_num_range)
        else:
            q_values = dqn(state_tensor)
            action = torch.argmax(q_values).item()

        next_state, reward, done = env.step(action)
        total_reward += reward

        next_state_vector = np.zeros(lotto_num_range)
        for num in next_state:
            next_state_vector[num - 1] = 1
        next_state_tensor = torch.FloatTensor(next_state_vector).unsqueeze(0)

        target = reward + gamma * torch.max(dqn(next_state_tensor))
        q_values = dqn(state_tensor)

        if action >= q_values.shape[1]:
            action = q_values.shape[1] - 1

        loss = loss_fn(q_values[0, action], target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state

    print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")

# 학습 완료 후 예측된 10개의 번호 조합 출력
predicted_tickets = predict_numbers(dqn, num_count=6, ticket_count=10)
for idx, ticket in enumerate(predicted_tickets, start=1):
    print(f"Predicted Lotto Ticket {idx}: {ticket}")


Episode 1/1000, Total Reward: -75.79999999999994
Episode 2/1000, Total Reward: -62.40000000000063
Episode 3/1000, Total Reward: -171.7999999999945
Episode 4/1000, Total Reward: -23.30000000000009
Episode 5/1000, Total Reward: -78.69999999999978
Episode 6/1000, Total Reward: -53.900000000000524
Episode 7/1000, Total Reward: -101.29999999999846
Episode 8/1000, Total Reward: -39.30000000000032
Episode 9/1000, Total Reward: -31.90000000000019
Episode 10/1000, Total Reward: -66.70000000000046
Episode 11/1000, Total Reward: -141.89999999999617
Episode 12/1000, Total Reward: -42.50000000000038
Episode 13/1000, Total Reward: -26.200000000000113
Episode 14/1000, Total Reward: -164.59999999999482
Episode 15/1000, Total Reward: -41.800000000000345
Episode 16/1000, Total Reward: -93.09999999999891
Episode 17/1000, Total Reward: -152.0999999999956
Episode 18/1000, Total Reward: -61.600000000000655
Episode 19/1000, Total Reward: -56.90000000000058
Episode 20/1000, Total Reward: -138.7999999999964
Ep

In [ ]:
import numpy as np
import pandas as pd